In [1]:
import numpy as np
import pandas as pd


# Replace 'your_dataset.csv' with the actual file path on your Google Drive
file_path = 'C:/Users/Roshan/Downloads/Neural/white_wine_dataset2.csv'

# Load the dataset
df = pd.read_csv(file_path)
X = df.drop('quality', axis=1)  # Adjust 'target_column' to the actual column name of your target variable
y = df['quality']

# Now you can use 'df' as your dataset
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

def softmax_derivative(x):
    s = softmax(x)
    return s * (1 - s)

def one_hot_encode(labels):
    num_classes = len(np.unique(labels))
    encoded = np.zeros((len(labels), num_classes))
    for i, label in enumerate(labels):
        encoded[i, label] = 1
    return encoded

def initialize_weights(input_size, hidden_size, output_size):
    np.random.seed(42)
    weights_input_hidden = 2 * np.random.random((input_size, hidden_size)) - 1
    weights_hidden_output = 2 * np.random.random((hidden_size, output_size)) - 1
    return weights_input_hidden, weights_hidden_output

def rmsprop_update(parameters, gradients, cache, learning_rate=0.001, decay_rate=0.9, epsilon=1e-8):
    for param, grad, accum in zip(parameters, gradients, cache):
        accum = decay_rate * accum + (1 - decay_rate) * grad**2
        param -= (learning_rate / np.sqrt(accum + epsilon)) * grad
    return parameters

def train_neural_network(X, y, hidden_size, output_size, epochs=10000, learning_rate=0.01):
    input_size = X.shape[1]

    weights_input_hidden, weights_hidden_output = initialize_weights(input_size, hidden_size, output_size)

    # RMSprop parameters
    accum_input_hidden = np.zeros_like(weights_input_hidden)
    accum_hidden_output = np.zeros_like(weights_hidden_output)

    for epoch in range(epochs):
        # Forward pass
        hidden_layer_input = np.dot(X, weights_input_hidden)
        hidden_layer_output = sigmoid(hidden_layer_input)

        output_layer_input = np.dot(hidden_layer_output, weights_hidden_output)
        predicted_output = softmax(output_layer_input)

        # Calculate errors
        output_error = y - predicted_output
        hidden_layer_error = output_error.dot(weights_hidden_output.T)

        # Backpropagation
        output_delta = output_error * softmax_derivative(predicted_output)
        hidden_layer_delta = hidden_layer_error * sigmoid_derivative(hidden_layer_output)

        # Update weights using RMSprop
        weights_hidden_output_gradients = hidden_layer_output.T.dot(output_delta)
        weights_input_hidden_gradients = X.T.dot(hidden_layer_delta)

        weights_hidden_output = rmsprop_update(
            weights_hidden_output, [weights_hidden_output_gradients], [accum_hidden_output], learning_rate
        )
        weights_input_hidden = rmsprop_update(
            weights_input_hidden, [weights_input_hidden_gradients], [accum_input_hidden], learning_rate
        )

        if epoch % 1000 == 0:
            loss = -np.sum(y * np.log(predicted_output)) / len(y)
            print(f"Epoch: {epoch}, Loss: {loss}")

    return weights_input_hidden, weights_hidden_output

# Example usage:
# Assume you have a dataset with input X and target y
# X = pd.DataFrame(...)
# y = pd.Series(...)

# Convert dataframes to numpy arrays
X = X.values
y = y.values

# Normalize data if needed
X = (X - X.mean()) / X.std()

# Map labels to integers
label_mapping = {label: idx for idx, label in enumerate(np.unique(y))}
y_mapped = np.array([label_mapping[label] for label in y])

# One-hot encode labels
y_one_hot = one_hot_encode(y_mapped)

# Add bias term to input
X = np.c_[X, np.ones(X.shape[0])]

# Train the neural network
hidden_size = 4
output_size = len(np.unique(y))
trained_weights_input_hidden, trained_weights_hidden_output = train_neural_network(X, y_one_hot, hidden_size, output_size)


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/Roshan/Downloads/Neural/white_wine_dataset2.csv'